# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [1]:
%load_ext autoreload
%autoreload 2
import sys

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [327]:
import pandas as pd
import numpy as np
import cufflinks as cf; cf.go_offline()

In [3]:
import os

print("Path at terminal when executing this file")
print(os.getcwd() + "\n")

Path at terminal when executing this file
C:\Users\yibar\Python_ML_2023\Exercises\Final Project



In [4]:
pd.options.display.max_colwidth = 1000

In [5]:
wine_reviews = pd.read_csv("winemag-data-130k-v2.csv", index_col = 0)
wine_reviews.shape

(129971, 13)

In [6]:
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks


In [7]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [8]:
#set points column to float
wine_reviews.points = wine_reviews.points.astype('float64')

In [9]:
#Colunm 'taster_twitter_handle' has no added value, same as taster_name
wine_reviews.drop('taster_twitter_handle', axis=1, inplace=True)

In [10]:
wine_reviews.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129908,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and pomegranate, this refreshing, light-bodied quaff is infinitely enjoyable, both on its own or at the table. It continues to expand on the palate into an increasing array of fresh flavors, finishing in cherry and orange.",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,92506,37979,Reserve,2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,120975.0,NaN,NaN,NaN,35.363389,41.022218,4.0,17.0,25.0,42.0,3300.0
province,129908,425,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,108724,1229,Napa Valley,4480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,50511,17,Central Coast,11065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,103727,19,Roger Voss,25514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,129971,118840,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County),11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Fill null values

In [12]:
wine_reviews.isnull().sum().sort_values(ascending=False)

region_2       79460
designation    37465
taster_name    26244
region_1       21247
price           8996
country           63
province          63
variety            1
description        0
points             0
title              0
winery             0
dtype: int64

In [13]:
# When country is null, try to fill from the counry of the same winery from the dataset
null_winery = pd.Series(wine_reviews[wine_reviews.country.isnull()].winery)

In [14]:
ds_winery_country = wine_reviews.loc[wine_reviews.winery.isin(null_winery.values), ['winery','country']] \
    .sort_values(['winery', 'country']).fillna(method = 'ffill').drop_duplicates()
ds_winery_country.columns = ['winery', 'country1']
ds_winery_country.head()

,winery,country1
119787,Amiran Vepkhvadze,NaN
13062,Bachelder,Canada
34853,Bartho Eksteen,South Africa
2789,Barton & Guestier,France
129590,Büyülübağ,France


In [15]:
wine_reviews = wine_reviews.merge(ds_winery_country, on = 'winery', how = 'left')
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,country1
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87.0,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,NaN
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87.0,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,NaN
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,NaN
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,NaN
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,NaN


In [16]:
wine_reviews.loc[wine_reviews.country.isnull() & wine_reviews.country1.notnull() ,'country'] = \
    wine_reviews.loc[wine_reviews.country.isnull() & wine_reviews.country1.notnull() ,'country1']

wine_reviews.drop('country1', axis=1, inplace=True)

In [17]:
wine_reviews.loc[wine_reviews.country.isnull()]

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery
119787,NaN,"This deep red-violet wine has a nose of blackberry, cassis and roast meat. With strong tannins buffered by equally strong acidity, it has flavors of cassis, black cherry, flame broiled meat, leather and violet. Rigid tannins are subdued by a bright burst of cranberry on the finish.",NaN,90.0,30.0,NaN,NaN,NaN,Mike DeSimone,Amiran Vepkhvadze 2014 Otskhanuri Sapere,Otskhanuri Sapere,Amiran Vepkhvadze


In [18]:
#Only one row left without country. The correct country for The last winery is Georgia, according to google...
wine_reviews.loc[wine_reviews.country.isnull(), 'country'] = 'Georgia'

In [19]:
wine_reviews.isnull().sum().sort_values(ascending=False)

region_2       79460
designation    37465
taster_name    26244
region_1       21247
price           8996
province          63
variety            1
country            0
description        0
points             0
title              0
winery             0
dtype: int64

In [20]:
#Replace all text null values by "Unknown" word
wine_reviews.loc[:, wine_reviews.columns != 'price'] = wine_reviews.loc[:, wine_reviews.columns != 'price'].fillna(value = 'Unknown')

In [21]:
wine_reviews.isnull().sum().sort_values(ascending=False)

price          8996
country           0
description       0
designation       0
points            0
province          0
region_1          0
region_2          0
taster_name       0
title             0
variety           0
winery            0
dtype: int64

In [22]:
#Replace null prices according to avarage price of:
#1. winery / variety / designation
#2. winery / variety
#3. winery 
#4. total avarage
wine_reviews['price'].fillna(wine_reviews.groupby(['winery', 'variety', 'designation'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.groupby(['winery', 'variety'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.groupby(['winery'])['price'].transform('mean'), inplace = True)
wine_reviews['price'].fillna(wine_reviews.price.mean(), inplace = True)

In [23]:
wine_reviews['price'].isnull().sum()

0

In [24]:
wine_reviews.describe()

,points,price
count,129971.000000,129971.000000
mean,88.447138,36.202300
std,3.039730,45.746672
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,26.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


In [25]:
def extact_year(title):
    for word in title.split():
        if word.isdigit():
            if 1900 <= int(word) <= 2020:
                return word
        elif word == 'NV':
            return word
    return None

In [26]:
#add Year Column
wine_reviews['year'] = wine_reviews['title'].apply(lambda i: extact_year(i))
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,2012


In [27]:
wine_reviews.isnull().sum().sort_values(ascending=False)

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
region_2       0
taster_name    0
title          0
variety        0
winery         0
year           0
dtype: int64

In [28]:
wine_reviews['year'].drop_duplicates().sort_values()

9728      1912
7986      1929
107854    1934
41441     1935
90583     1941
117205    1945
51898     1947
82827     1952
78299     1957
78312     1961
52636     1963
12860     1964
45034     1965
78310     1966
16331     1967
46010     1968
36196     1969
45000     1973
90603     1974
12530     1976
45010     1978
31417     1980
60004     1982
16011     1983
78311     1984
54667     1985
60384     1986
31425     1987
18792     1988
33974     1989
18791     1990
370       1991
6050      1992
33976     1993
6037      1994
2040      1995
378       1996
375       1997
4731      1998
367       1999
366       2000
132       2001
243       2002
120       2003
119       2004
137       2005
123       2006
22        2007
50        2008
24        2009
16        2010
1         2011
4         2012
0         2013
59        2014
61        2015
77        2016
1618      2017
63          NV
Name: year, dtype: object

In [29]:
#Adding log of price to reduce outlier prices
#Therefore, we can remove it.
wine_reviews['price_log'] = np.log(wine_reviews.price)
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year,price_log
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,2.944439
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,2.708050
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,2.639057
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,2013,2.564949
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,2012,4.174387


In [30]:
wine_reviews.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129971,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and pomegranate, this refreshing, light-bodied quaff is infinitely enjoyable, both on its own or at the table. It continues to expand on the palate into an increasing array of fresh flavors, finishing in cherry and orange.",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,129971,37980,Unknown,37465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,129971.0,NaN,NaN,NaN,36.2023,45.746672,4.0,17.0,26.0,42.0,3300.0
province,129971,426,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,129971,1230,Unknown,21247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,129971,18,Unknown,79460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,129971,20,Unknown,26244,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,129971,118840,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County),11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#Remove most non-alphanumeric characters from the description
#wine_reviews.description = wine_reviews.description.str.replace(r'[^a-zA-Z0-9,.;''"\- ]+', '', regex=True)

In [32]:
wine_reviews['desc_len'] = wine_reviews['description'].str.split().str.len()
wine_reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,year,price_log,desc_len
0,Italy,"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.",Vulkà Bianco,87.0,19.0,Sicily & Sardinia,Etna,Unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,2.944439,24
1,Portugal,"This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's already drinkable, although it will certainly be better from 2016.",Avidagos,87.0,15.0,Douro,Unknown,Unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,2.708050,38
2,US,"Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.",Unknown,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,2.639057,28
3,US,"Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,Unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore),Riesling,St. Julian,2013,2.564949,33
4,US,"Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country wine, it's a good companion to a hearty winter stew.",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child Block Pinot Noir (Willamette Valley),Pinot Noir,Sweet Cheeks,2012,4.174387,41


In [33]:
wine_reviews.describe(include = 'all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
country,129971,43,US,54504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
description,129971,119955,"Seductively tart in lemon pith, cranberry and pomegranate, this refreshing, light-bodied quaff is infinitely enjoyable, both on its own or at the table. It continues to expand on the palate into an increasing array of fresh flavors, finishing in cherry and orange.",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
designation,129971,37980,Unknown,37465,NaN,NaN,NaN,NaN,NaN,NaN,NaN
points,129971.0,NaN,NaN,NaN,88.447138,3.03973,80.0,86.0,88.0,91.0,100.0
price,129971.0,NaN,NaN,NaN,36.2023,45.746672,4.0,17.0,26.0,42.0,3300.0
province,129971,426,California,36247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_1,129971,1230,Unknown,21247,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region_2,129971,18,Unknown,79460,NaN,NaN,NaN,NaN,NaN,NaN,NaN
taster_name,129971,20,Unknown,26244,NaN,NaN,NaN,NaN,NaN,NaN,NaN
title,129971,118840,Gloria Ferrer NV Sonoma Brut Sparkling (Sonoma County),11,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
wine_reviews[wine_reviews.desc_len > 100][['description','points']]

,description,points
1557,"A stupendous Pinot Noir, showing how beautifully this vineyard performs in the remote Sonoma Coast above Fort Ross. Williams Selyem's last few vintages of Precious Mountain have been extraordinary, but this 2009 tops them all. The wine is as light as a feather in the mouth, modest in alcohol and delicately feminine, like pure satin. However, few Pinot Noirs in the world achieve this level of concentration. Wild raspberries and cherries, cola, sleek minerals, dusty baking spices and smoky oak combine for flavors that are utterly delicious. After all that, it possesses an indefinable attribute of fascination. Will age well for a long time, if you can keep your hands off it. Now–2021.",99.0
3077,"Though it's a huge tannic monster right now, this Bordeaux blend, a first release and new brand name from Steven Kent Winery, is an ode to Steven Kent Mirassou's family's long lineage in California winemaking and the roots of Livermore's history in wine as well. A majority Cabernet (75%), with all the lush cassis and black cherry one would hope for, the wine shows nicely honed herbal notes from its 15% Cab Franc in the middle, with a little touch of spice on the finish coming from 5% Petit Verdot (3% Malbec and 2% Merlot round out the blend). Give this one time and it will surely reward.",94.0
9899,"Deliciously heady notes of peach and chamomile immediately hit the nose, promising ripeness and aromatic riches. The palate does not disappoint, releasing even more of those wonderfully balm-like, aromatic flavors. It brightens them with wondrous lemon freshness and boosts them with a poised dose of residual sweetness. It's an unusual wine that does not really fit any category but sits in a delicious aromatic off-dry class of its own. The best option with this probably is cheese, but it will serve well as a philosopher's wine, savored on its own. It has enough depth and beauty to nourish sustained thought. Drink 2018–2035.",96.0
9912,"Only 95 cases were made of this Bordeaux-style blend of 43% Cabernet Sauvignon, 33% Malbec and 24% Cabernet Franc. It's a stellar example of the high-end and highly cellarable wines that can come out of South Africa, oozing with depth, concentration and complexity, all in a harmonious, balanced package. It's dark and brooding right off the bat, boasting firm, tight blackberry, cherry and cassis fruit that's coupled with hints of raspberry sauce, pressed violets and baking spice. The palate is strong and packed with black fruit flavors that lead into a long, evolving finish of char, espresso bean, cigar and woodspice. Firmly structured and robust in feel, try from 2022–2028, though it could mature well even beyond.",95.0
19577,"Fermented to a plus-size 15.4% alcohol, this rare wine from one of Washington's oldest and best Cabernet vineyards shows what a gifted winemaker can do with great fruit. The play of flavors, mixing red and purple fruits, toasted nuts, mocha, caramel, bark and earth, is exceptional. The detail is extraordinary for a wine with such high alcohol. The acids and tannins are seamless and fine, and the flavors extend into an unusual finish that suggests menthol and eucalyptus – not generally a character found in Champoux fruit. All in all it's a wonderful voyage, and one of a handful of essential Washington wines.",95.0
26672,"This very fine Cabernet wants a little time in the cellar. Right now, it's tight in tannins, with some acidic bitterness in the finish. The flavors are of black currants and smoky new oak. The Morisoli Vineyard has been home to very good, ageable bottlings from the likes of Sequoia Grove and Elyse, but in the last few years, Meander has expressed its terroir best. Try after 2012.This very fine Cabernet wants a little time in the cellar. Right now, it's tight in tannins, with some acidic bitterness in the finish. The flavors are of black currants and smoky new oak. The Morisoli Vineyard has been home to very good, ageable bottlings from the likes of

In [326]:
wine_reviews.to_csv("wine_reviews.csv")